In [1]:
import os
import torch
import numpy as np
import pandas as pd
import pdb
import time
import pickle
import logging
import matplotlib.pyplot as plt
from utilities import Mapping


import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics 

In [2]:
data_path='data/'
save_path='save/'

bsz=128
cuda=True
device=0
torch.cuda.set_device(device)

num_gene_compressed_drug=64
num_gene_compressed_cell=128

#isClassification=True #False for regression task
syn_threshold=30
ri_threshold=50

log_interval=100
epochs=10

## Load data

In [3]:
#drug pair, cell, scoers
df=pickle.load(open(data_path+'summary_mean.p', 'rb'))
codes=pickle.load(open(data_path+'codes.p', 'rb'))

In [4]:
df.head()

,drug_row,drug_col,cell_line_name,study_id,ri_row,ri_col,synergy_loewe
0,3865,705,0,4.0,-21.0794,17.392589,4.436431
1,3817,218,1,6.0,-20.0430,25.595000,-44.555935
2,3817,432,1,6.0,-9.7760,29.111000,-37.189720
3,218,218,2,12.0,6.7970,6.964000,1.283298
4,218,218,3,12.0,11.5280,7.190000,-3.028745


In [5]:
# Drug's external features
drug_features=pickle.load(open(data_path+'drug_features.p', 'rb'))
# drug_features = pd.read_csv(data_path+'drug_features.csv')

In [6]:
drug_features

,smiles,fps,gene_id
id,,,
0,"[47, 27, 1, 4, 13, 30, 47, 31, 30, 47, 31, 47,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",[10256]
1,"[47, 1, 6, 47, 20, 47, 30, 47, 30, 20, 1, 6, 3...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",[7880]
2,"[47, 6, 47, 1, 30, 47, 47, 1, 6, 47, 47, 47, 1...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",[]
3,"[47, 47, 30, 47, 31, 47, 47, 6, 47, 1, 17, 47,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[22597, 3305, 16393, 5169, 1613]"
4,"[47, 47, 6, 47, 20, 47, 47, 20, 47, 30, 47, 30...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",[]
...,...,...,...
4144,"[47, 1, 30, 47, 31, 47, 47, 20, 47, 47, 30, 20...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",[1769]
4145,"[47, 47, 30, 47, 6, 20, 47, 47, 20, 47, 47, 20...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",[]
4146,"[47, 6, 47, 1, 30, 47, 27, 47, 16, 16, 10, 13,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",[]


In [7]:
# Cell's external features
cell_features=pickle.load(open(data_path+'cell_features.p', 'rb'))

In [8]:
num_celllines= len(codes['cell'].idx2item)
num_drugs=len(codes['drugs'].idx2item)

num_genes = len(codes['gene'].idx2item)
num_tissue = len(codes['tissue'].idx2item)
num_disease = len(codes['disease'].idx2item)

num_drug_fp=len(drug_features.loc[0,'fps'])
max_drug_sm_len = drug_features['smiles'].apply(lambda x: len(x)).max()

# Gene compression

## Dataset and Dataloader

In [9]:
class DrugTargetDataset(Dataset):
    def __init__(self, drug_features):
        self.drug_features = drug_features
    def __len__(self):
        return len(self.drug_features)
    def __getitem__(self,idx):
        gene_ids=self.drug_features.loc[idx, 'gene_id']
        genes=np.zeros(num_genes)
        genes[gene_ids]=1
        
        return genes

# class DrugTargetDataset(Dataset):
#     def __init__(self, drug_features):
#         self.drug_features = drug_features

#     def __len__(self):
#         return len(self.drug_features)

#     def __getitem__(self, idx):
#         if idx not in self.drug_features.index:
#             gene_ids = self.drug_features.loc[idx-1, 'gene_id']
#             genes = np.zeros(num_genes)
#             genes[gene_ids] = 1
#             return genes

#         try:
#             gene_ids = self.drug_features.loc[idx, 'gene_id']
#             genes = np.zeros(num_genes)
#             genes[gene_ids] = 1
#             return genes
#         except KeyError as e:
#             print(f"Skipping index {idx} due to KeyError: {e}")
#             return None  # Return None or consider raising an exception

    
class CellGeneDataset(Dataset):
    def __init__(self, cell_features):
        self.cell_features = cell_features
    def __len__(self):
        return len(self.cell_features)
    def __getitem__(self,idx):
        gene_ids=self.cell_features.loc[idx,'gene_id']
        genes = np.zeros(num_genes)
        for key,value in gene_ids.items():
            genes[key]=value
            
        return genes
            

In [10]:
drug_target_dataset = DrugTargetDataset(drug_features)

In [11]:
# drug_target_dataset[2115]
# drug_features.iloc[2114]
# drug_target_dataset[3904]
drug_features

,smiles,fps,gene_id
id,,,
0,"[47, 27, 1, 4, 13, 30, 47, 31, 30, 47, 31, 47,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",[10256]
1,"[47, 1, 6, 47, 20, 47, 30, 47, 30, 20, 1, 6, 3...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",[7880]
2,"[47, 6, 47, 1, 30, 47, 47, 1, 6, 47, 47, 47, 1...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",[]
3,"[47, 47, 30, 47, 31, 47, 47, 6, 47, 1, 17, 47,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[22597, 3305, 16393, 5169, 1613]"
4,"[47, 47, 6, 47, 20, 47, 47, 20, 47, 30, 47, 30...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",[]
...,...,...,...
4144,"[47, 1, 30, 47, 31, 47, 47, 20, 47, 47, 30, 20...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",[1769]
4145,"[47, 47, 30, 47, 6, 20, 47, 47, 20, 47, 47, 20...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",[]
4146,"[47, 6, 47, 1, 30, 47, 27, 47, 16, 16, 10, 13,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",[]


In [12]:
#Two layers of fully connected layers
class FC2(nn.Module):
    def __init__(self, in_features, out_features, dropout):
        super(FC2, self).__init__()
        
        self.bn = nn.BatchNorm1d(in_features)
        self.fc1 = nn.Linear(in_features, int(in_features/2))
        self.fc2 = nn.Linear(int(in_features/2),out_features)
        self.dropout= nn.Dropout(dropout)
                
    def forward(self, x):
        x = self.bn(x)
        x = self.dropout(x)
        x = self.dropout(F.relu(self.fc1(x)))
        x = self.fc2(x)
        
        return x


## Compress gene features

In [13]:
class GeneCompressor(nn.Module):
    def __init__(self, num_in, num_out, dropout=0.1):
        super(GeneCompressor, self).__init__()
        self.dropout=dropout
        self.encoder=nn.Linear(num_in, num_out)
        self.decoder=nn.Linear(num_out,num_in)

    def _encoder(self,x):
        return F.dropout(F.relu(self.encoder(x)), self.dropout, training=self.training)
    
    def _decoder(self,x):
        return F.dropout(self.decoder(x), self.dropout, training=self.training)
    
    def forward(self, x):
        x=self._encoder(x)
        x=self._decoder(x)
        return x

In [14]:
# def geneCompressing(data_loader, num_gene_compressed, noise_weight=0.2, epochs=20, log_interval=10):
#     geneCompressor = GeneCompressor(num_genes, num_out=num_gene_compressed, dropout=0.1)
#     if cuda:
#         geneCompressor = geneCompressor.cuda()
#     criterion = nn.MSELoss()
#     optimizer = optim.Adam(geneCompressor.parameters())

#     for epoch in range(1, epochs + 1):
#         geneCompressor.train()
#         total_loss = 0
#         start_time = time.time()
#         for iteration, gene in enumerate(data_loader):
#             try:
#                 gene = Variable(gene).float()
#                 noise = noise_weight * torch.randn(gene.shape)

#                 if cuda:
#                     gene = gene.cuda()
#                     noise = noise.cuda()

#                 optimizer.zero_grad()
#                 output = geneCompressor(gene + noise)
#                 loss = criterion(output, gene)
#                 loss.backward()
#                 optimizer.step()
#                 total_loss += loss.data

#                 if iteration % log_interval == 0 and iteration > 0:
#                     cur_loss = total_loss.item() / log_interval
#                     elapsed = time.time() - start_time
#                     print('| epoch {:3d} | {:5d}/{:5d} batches | ms/batch {:5.2f} | loss {:8.5f}'.format(
#                         epoch, iteration, int(len(data_loader.dataset) / data_loader.batch_size), elapsed * 1000 / log_interval, cur_loss))
#                     total_loss = 0
#                     start_time = time.time()

#             except KeyError as e:
#                 print(f"Skipping batch due to KeyError: {e}")
#                 continue

#     return geneCompressor


def geneCompressing(data_loader,num_gene_compressed, noise_weight=0.2, epochs=20, log_interval=10 ):
    #model
    geneCompressor=GeneCompressor(num_genes, num_out=num_gene_compressed, dropout=0.1)
    if cuda: 
        geneCompressor=geneCompressor.cuda()
    criterion=nn.MSELoss()
    optimizer=optim.Adam(geneCompressor.parameters())

    for epoch in range(1,epochs+1):
        #train
        geneCompressor.train()
        total_loss=0
        start_time=time.time()
        for iteration, gene in enumerate(data_loader):
            gene=Variable(gene).float()
            noise=noise_weight*torch.randn(gene.shape)

            if cuda:
                gene=gene.cuda()
                noise=noise.cuda()
            optimizer.zero_grad()
            output=geneCompressor(gene+noise)
            loss=criterion(output,gene)
            loss.backward()
            optimizer.step()
            total_loss += loss.data
            if iteration % log_interval == 0 and iteration > 0:
                cur_loss = total_loss.item() / log_interval
                elapsed = time.time() - start_time
                print('| epoch {:3d} | {:5d}/{:5d} batches | ms/batch {:5.2f} | loss {:8.5f}'.format(epoch, iteration, int(len(data_loader)/bsz), elapsed * 1000/log_interval, cur_loss))
                total_loss = 0
                start_time = time.time()
#         #test
#         geneCompressor.eval()
#         total_loss=0
#         start_time=time.time()
#         with torch.no_grad():
#             for iteration, gene in enumerate(test_data_loader):
#                 gene=Variable(gene).float()
#                 if cuda:
#                     gene=gene.cuda(device)
#                 output=geneCompressor(gene)
#                 loss=criterion(output,gene)
#                 total_loss += loss.data
#             print(total_loss.item()/iteration)
    return geneCompressor


In [15]:
#drug's target gene data
drugGeneDataset=DrugTargetDataset(drug_features)
drugGeneDataset_loader = DataLoader(drugGeneDataset, batch_size=64, shuffle=True)
#learn
drugGeneCompressor=geneCompressing(drugGeneDataset_loader, num_gene_compressed_drug)
#save
drugGeneCompressor.eval()
drugGeneCompressed=np.array([drugGeneCompressor.cpu()._encoder(torch.FloatTensor(drugGeneDataset[d])).data.numpy() for d in range(num_drugs)])
torch.save(drugGeneCompressor.state_dict(), data_path+'drugGeneCompressor.p')
pickle.dump(drugGeneCompressed, open(data_path+'drugGeneCompressed.p', 'wb'))

| epoch   1 |    10/    0 batches | ms/batch 32.90 | loss  0.00778
| epoch   1 |    20/    0 batches | ms/batch 16.83 | loss  0.00507
| epoch   1 |    30/    0 batches | ms/batch 16.77 | loss  0.00501
| epoch   1 |    40/    0 batches | ms/batch 17.85 | loss  0.00505
| epoch   1 |    50/    0 batches | ms/batch 16.62 | loss  0.00480
| epoch   1 |    60/    0 batches | ms/batch 18.26 | loss  0.00440
| epoch   2 |    10/    0 batches | ms/batch 18.01 | loss  0.00423
| epoch   2 |    20/    0 batches | ms/batch 15.17 | loss  0.00320
| epoch   2 |    30/    0 batches | ms/batch 20.04 | loss  0.00289
| epoch   2 |    40/    0 batches | ms/batch 16.97 | loss  0.00245
| epoch   2 |    50/    0 batches | ms/batch 16.95 | loss  0.00207
| epoch   2 |    60/    0 batches | ms/batch 17.93 | loss  0.00174
| epoch   3 |    10/    0 batches | ms/batch 18.01 | loss  0.00150
| epoch   3 |    20/    0 batches | ms/batch 17.00 | loss  0.00114
| epoch   3 |    30/    0 batches | ms/batch 17.17 | loss  0.0

In [16]:
drugGeneCompressed=pickle.load(open(data_path+'drugGeneCompressed.p', 'rb'))
drugGeneCompressed=torch.FloatTensor(drugGeneCompressed)

In [17]:

#cell line's gene expression data
cellGeneDataset=CellGeneDataset(cell_features)
cellGeneDataset_loader = DataLoader(cellGeneDataset, batch_size=64, shuffle=True)
#learn
cellGeneCompressor=geneCompressing(cellGeneDataset_loader,num_gene_compressed_cell, noise_weight=0.01, log_interval=1 )
#save
cellGeneCompressor.eval()
cellGeneCompressed=np.array([cellGeneCompressor.cpu()._encoder(torch.FloatTensor(cellGeneDataset[d])).data.numpy() for d in range(num_celllines)])
torch.save(cellGeneCompressor.state_dict(), data_path+'cellGeneCompressor.p')
pickle.dump(cellGeneCompressed, open(data_path+'cellGeneCompressed.p', 'wb'))


| epoch   1 |     1/    0 batches | ms/batch 158.72 | loss  2.17674
| epoch   2 |     1/    0 batches | ms/batch 153.25 | loss  3.03503
| epoch   3 |     1/    0 batches | ms/batch 153.40 | loss  2.97886
| epoch   4 |     1/    0 batches | ms/batch 152.31 | loss  2.57283
| epoch   5 |     1/    0 batches | ms/batch 156.91 | loss  2.21477
| epoch   6 |     1/    0 batches | ms/batch 161.93 | loss  2.07943
| epoch   7 |     1/    0 batches | ms/batch 162.50 | loss  1.98452
| epoch   8 |     1/    0 batches | ms/batch 164.29 | loss  1.91247
| epoch   9 |     1/    0 batches | ms/batch 161.50 | loss  1.76785
| epoch  10 |     1/    0 batches | ms/batch 158.24 | loss  1.76761
| epoch  11 |     1/    0 batches | ms/batch 160.66 | loss  1.63839
| epoch  12 |     1/    0 batches | ms/batch 166.66 | loss  1.58713
| epoch  13 |     1/    0 batches | ms/batch 161.15 | loss  1.57965
| epoch  14 |     1/    0 batches | ms/batch 164.41 | loss  1.53907
| epoch  15 |     1/    0 batches | ms/batch 152

In [18]:
cellGeneCompressed=pickle.load(open(data_path+'cellGeneCompressed.p', 'rb'))
cellGeneCompressed=torch.FloatTensor(cellGeneCompressed)

# Synergy prediction

## Train/test split in cross or external validation

In [19]:
def get_cell_of_interest(tissues):
    tissues_of_interests = [codes['tissue'].item2idx[minor_tissue] for minor_tissue in tissues]
    cell_of_interest = cell_features.index[cell_features['tissue_id'].isin(tissues_of_interests)].tolist()
    return cell_of_interest

Train/test for general model

In [20]:
minor_tissues=['bone', 'prostate' ]
cell_of_interest = get_cell_of_interest(minor_tissues)
df_tissue_of_interest = df.loc[df['cell_line_name'].isin(cell_of_interest),:]
df_all = df.drop(df_tissue_of_interest.index)
#specific database
#df_all=df_all.loc[df_all['study_id']==3]
#cross validation
df_train, df_test = train_test_split(df_all, test_size=0.2) #cross validation
#external validation
#df_train=df_all.loc[df_all['study_id']==3] 3: 'ALMANAC'
#df_test=df_all.loc[df_all['study_id']==1] 1: 'ONEIL'

Train/test for bone

In [21]:
_df_bone = df.loc[df['cell_line_name'].isin(get_cell_of_interest(['bone'])),:]
# Cross validation
_df_train_bone, _df_test_bone = train_test_split(_df_bone, test_size=0.2, random_state=1)
# External validation
#_df_train_bone=_df_bone.loc[_df_bone['study_id']!=9]
#_df_test_bone= _df_bone.loc[_df_bone['study_id']==9]

Train/test for prostate

In [22]:
_df_prostate= df.loc[df['cell_line_name'].isin(get_cell_of_interest(['prostate'])),:]
#Cross validation
_df_train_prostate, _df_test_prostate = train_test_split(_df_prostate, test_size=0.2, random_state=1)
# External validation
#_df_train_prostate=_df_prostate.loc[_df_prostate['study_id']!=1]
#_df_test_prostate=_df_prostate.loc[_df_prostate['study_id']==1]

## Dataset and Dataloader

In [23]:
class DrugCombDataset(Dataset):
    def __init__(self, df, drug_features, cell_features):
        self.df = df
        self.drug_features = drug_features
        self.cell_features = cell_features
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        
        d1 = self.df.iloc[idx, 0]
        d2 = self.df.iloc[idx, 1]
        cell = self.df.iloc[idx,2]
        ri_d1 = 1.0 if self.df.iloc[idx,3] >ri_threshold else 0
        ri_d2 = 1.0 if self.df.iloc[idx,4] >ri_threshold else 0
        syn = 1.0 if self.df.iloc[idx, 5] >syn_threshold else 0
        
        
        #external features
        d1_fp = np.array(self.drug_features.loc[d1, 'fps'])
        d1_sm = self.drug_features.loc[d1, 'smiles']
        d1_sm = np.pad(d1_sm, pad_width=(0, max_drug_sm_len-len(d1_sm)), mode='constant', constant_values=0)
        d1_gn=drugGeneCompressed[d1]
        
        d2_fp = np.array(self.drug_features.loc[d2, 'fps'])
        d2_sm = self.drug_features.loc[d2, 'smiles']
        d2_sm = np.pad(d2_sm, pad_width=(0, max_drug_sm_len-len(d2_sm)), mode='constant', constant_values=0)
        d2_gn=drugGeneCompressed[d2]
        
        c_ts = self.cell_features.loc[cell, 'tissue_id']
        c_ds = self.cell_features.loc[cell, 'disease_id']
        c_gn= cellGeneCompressed[cell]
        
        sample = {
            'd1': d1,
            'd1_fp': d1_fp,
            'd1_sm': d1_sm,
            'd1_gn': d1_gn,
            
            'd2': d2,
            'd2_fp': d2_fp,
            'd2_sm': d2_sm,
            'd2_gn': d2_gn,
            
            'cell': cell,
            'c_ts': c_ts,
            'c_ds': c_ds, #missing -1
            'c_gn': c_gn,
            
            'ri_d1': ri_d1,
            'ri_d2': ri_d2,
            'syn': syn
        }
        
        return sample

## Load dataset

General model

In [24]:
train = DrugCombDataset(df_train, drug_features, cell_features)
train_loader = DataLoader(train, batch_size=bsz, shuffle=True )
test = DrugCombDataset(df_test, drug_features, cell_features)
test_loader = DataLoader(test, batch_size=bsz, shuffle=True )

Bone

In [25]:
_train_bone = DrugCombDataset(_df_train_bone, drug_features, cell_features)
_train_loader_bone = DataLoader(_train_bone, batch_size=bsz, shuffle=True )
_test_bone = DrugCombDataset(_df_test_bone, drug_features, cell_features)
_test_loader_bone = DataLoader(_test_bone, batch_size=bsz, shuffle=False )

Prostate

In [26]:
_train_prostate = DrugCombDataset(_df_train_prostate, drug_features, cell_features)
_train_loader_prostate = DataLoader(_train_prostate, batch_size=bsz, shuffle=True )
_test_prostate = DrugCombDataset(_df_test_prostate, drug_features, cell_features)
_test_loader_prostate = DataLoader(_test_prostate, batch_size=bsz, shuffle=False )

# Prediction model

In [27]:
class DrugEncoder(nn.Module):
    def __init__(self,
                 num_drugs=num_drugs,
                 num_ID_emb=0,
                 num_drug_fp=num_drug_fp,
                 max_drug_sm_len=max_drug_sm_len,
                 num_gene = num_gene_compressed_drug,
                 num_comp_char=len(codes['mole'].idx2item),
                 fp_embed_sz = 32,
                 gene_embed_sz = int(num_gene_compressed_drug/2),
                 out_size=64,
                 dropout=0.3):
        super(DrugEncoder, self).__init__()
        
        self.dropout= dropout
        #DRUG
        #drug ID
        #self.embed_id = nn.Embedding(num_drugs, num_ID_emb)
        
        #compound ID
        self.embed_comp = nn.Embedding(num_comp_char, num_comp_char, padding_idx=0)#padding's idx=0
        #encoding compound
        self.encoderlayer = nn.TransformerEncoderLayer(d_model=num_comp_char, nhead=4)
        self.encoder = nn.TransformerEncoder(self.encoderlayer, num_layers=1)
        
        #fingerprint
        self.dense_fp = nn.Linear(num_drug_fp,fp_embed_sz)
        #gene
        self.dense_gene = nn.Linear(num_gene,gene_embed_sz)
        
        #depthwise for compound encoding
        self.conv = nn.Conv2d(1, 1, (1, num_comp_char), groups=1)
        
        #combined
        combined_sz = num_ID_emb+fp_embed_sz+max_drug_sm_len+gene_embed_sz
        self.FC2 = FC2(combined_sz, out_size, dropout)

    def forward(self, d_list):
        """
            id: bsz*1
            fp: bsz*num_drug_fp
            sm: bsz*max_drug_sm_len
        """
        id, fp, sm, gn = d_list
        
        sm = self.embed_comp(sm) #bsz*max_drug_sm_len*num_comp_char(embedding size)
        sm = self.encoder(sm)
        sm = self.conv(sm.unsqueeze(1)).squeeze()
        
        fp = F.relu(self.dense_fp(fp))
        gn = F.relu(self.dense_gene(gn))
        
        #combine
        x = torch.cat((fp, sm, gn),1) # bsz*[num_emb_id+num_drug_fp+max+drug_sm]
        x = self.FC2(x)
        
        return x
        

In [28]:
class CellEncoder(nn.Module):
    def __init__(self,
                 num_cells=num_celllines,
                 num_tissue=0,
                 num_disease=num_disease,
                 num_ID_emb=0,
                 gene_embed_sz=int(num_gene_compressed_cell/2),
                 num_gene=num_gene_compressed_cell,
                 out_size=64,
                 dropout=0.3):
        super(CellEncoder, self).__init__()
        
        self.dropout= dropout
        #cell ID
        #self.embed_id = nn.Embedding(num_cells, num_ID_emb)
        #cell tissue
        #self.embed_ts = nn.Embedding(num_tissue, num_tissue)
        #cell disease
        self.embed_ds = nn.Embedding(num_disease, num_disease, padding_idx=3)
        #gene
        self.dense_gene = nn.Linear(num_gene,gene_embed_sz)
        
        #combined
        combined_sz = num_ID_emb+num_tissue+num_disease+gene_embed_sz
        self.FC2 = FC2(combined_sz, out_size, dropout)
    
        
    def forward(self, c_list):
        """
            id: bsz*1
            fp: bsz*num_drug_fp
            sm: bsz*max_drug_sm_len
        """
        id, ts, ds, gn = c_list
        ds = F.relu(self.embed_ds(ds)) #bsz*num_diesaes
        
        gn = F.relu(self.dense_gene(gn)) #bsz*gene_embed_sz
        
        #combine
        x = torch.cat((ds, gn),1) # bsz*combined_sz
        x = self.FC2(x)

        return x
        

In [29]:
class Comb(nn.Module):
    def __init__(self, num_cells=num_celllines, 
                 num_drugs=num_drugs,
                 num_drug_fp=num_drug_fp,
                 max_drug_sm_len=max_drug_sm_len,
                num_comp_char=len(codes['mole'].idx2item),
                 num_ID_emb=0,
                 out_size=64,
                dropout=0.3):
        
        super(Comb, self).__init__()
        
        self.dropout=dropout    
        #drug
        self.drugEncoder = DrugEncoder()
        #cell
        self.cellEncoder = CellEncoder()
        #fc
        self.fc_syn = FC2(out_size*3, 1, dropout)
        self.fc_ri = FC2(out_size*2, 1, dropout)
        
    def forward(self, d1_list, d2_list, c_list):
        d1 = self.drugEncoder(d1_list)
        d2 = self.drugEncoder(d2_list)
        c = self.cellEncoder(c_list)
        
        syn = self.fc_syn(torch.cat((d1, d2, c),1))
        ri1 = self.fc_ri(torch.cat((d1,c),1))
        ri2 = self.fc_ri(torch.cat((d2,c),1))
        
        return syn, ri1, ri2
    

# Optimize the model

In [30]:
model = Comb()
if cuda:
    model = model.cuda()
#Regression
#criterion_mse = nn.MSELoss()
#Classification
criterion_bce = nn.BCEWithLogitsLoss()
optimizer = optim.Adagrad(model.parameters())


/home/blake/miniconda3/envs/synergy/lib/python3.8/site-packages/torch/nn/modules/transformer.py:286: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


In [31]:
#Training
def training(isAux, data_loader):
    model.train()
    total_loss = 0
    start_time = time.time()

    for iteration, sample in enumerate(data_loader):
        d1=Variable(sample['d1'])
        d1_fp = Variable(sample['d1_fp'].float())
        d1_sm = Variable(sample['d1_sm'])
        d1_gn = Variable(sample['d1_gn'].float())
        
        d2=Variable(sample['d2'])
        d2_fp = Variable(sample['d2_fp'].float())
        d2_sm = Variable(sample['d2_sm'])
        d2_gn = Variable(sample['d2_gn'].float())
        
        cell = Variable(sample['cell'])
        c_ts = Variable(sample['c_ts'])
        c_ds = Variable(sample['c_ds'])
        c_gn = Variable(sample['c_gn'].float())
        
        syn_true = Variable(sample['syn'].float())
        ri_d1=Variable(sample['ri_d1'].float())
        ri_d2=Variable(sample['ri_d2'].float())


        if cuda:
            d1=d1.cuda()
            d1_fp=d1_fp.cuda()
            d1_sm=d1_sm.cuda()
            d1_gn=d1_gn.cuda()
            
            d2=d2.cuda()
            d2_fp=d2_fp.cuda()
            d2_sm=d2_sm.cuda()
            d2_gn=d2_gn.cuda()
            
            cell=cell.cuda()
            c_ts=c_ts.cuda()
            c_ds=c_ds.cuda()
            c_gn=c_gn.cuda()
            
            syn_true=syn_true.cuda()
            ri_d1=ri_d1.cuda()
            ri_d2=ri_d2.cuda()

        optimizer.zero_grad()
        
        syn, ri1, ri2 = model((d1, d1_fp, d1_sm, d1_gn), (d2, d2_fp, d2_sm, d2_gn), (cell, c_ts, c_ds, c_gn) )
        
        if not isAux:
            loss = criterion_bce(syn, syn_true.view(-1,1))
        else:
            loss = criterion_bce(ri1, ri_d1.view(-1,1))+criterion_bce(ri2, ri_d2.view(-1,1))
            
        loss.backward()
        optimizer.step()
        total_loss += loss.data

        if iteration % log_interval == 0 and iteration > 0:
            cur_loss = total_loss.item() / log_interval
            elapsed = time.time() - start_time
            print('| epoch {:3d} | {:5d}/{:5d} batches | ms/batch {:5.2f} | loss {:8.5f}'.format(epoch, iteration, int(len(train_loader)/bsz), elapsed * 1000/log_interval, cur_loss))

            total_loss = 0
            start_time = time.time()


In [32]:
def evaluate(data_loader):
    model.eval()
    total_loss = 0
    total_loss_sen = 0

    
    #loss
    with torch.no_grad():
        for iteration, sample in enumerate(data_loader):
            d1=Variable(sample['d1'])
            d1_fp = Variable(sample['d1_fp'].float())
            d1_sm = Variable(sample['d1_sm'])
            d1_gn = Variable(sample['d1_gn'].float())

            d2=Variable(sample['d2'])
            d2_fp = Variable(sample['d2_fp'].float())
            d2_sm = Variable(sample['d2_sm'])
            d2_gn = Variable(sample['d2_gn'].float())

            cell = Variable(sample['cell'])
            c_ts = Variable(sample['c_ts'])
            c_ds = Variable(sample['c_ds'])
            c_gn = Variable(sample['c_gn'].float())

            syn_true = Variable(sample['syn'].float())
            ri_d1=Variable(sample['ri_d1'].float())
            ri_d2=Variable(sample['ri_d2'].float())


            if cuda:
                d1=d1.cuda()
                d1_fp=d1_fp.cuda()
                d1_sm=d1_sm.cuda()
                d1_gn=d1_gn.cuda()

                d2=d2.cuda()
                d2_fp=d2_fp.cuda()
                d2_sm=d2_sm.cuda()
                d2_gn=d2_gn.cuda()

                cell=cell.cuda()
                c_ts=c_ts.cuda()
                c_ds=c_ds.cuda()
                c_gn=c_gn.cuda()

                syn_true=syn_true.cuda()
                ri_d1=ri_d1.cuda()
                ri_d2=ri_d2.cuda()


            syn,ri1,ri2 = model((d1, d1_fp, d1_sm, d1_gn), (d2, d2_fp, d2_sm, d2_gn), (cell, c_ts,c_ds,c_gn) )
            loss = criterion_bce(syn, syn_true.view(-1,1))
            total_loss +=loss.data
            loss_sen = (criterion_bce(ri1, ri_d1.view(-1,1))+criterion_bce(ri2, ri_d2.view(-1,1)))/2
            total_loss_sen += loss_sen.data

        print('syn mse', total_loss.item()/(iteration+1))
        print('sen_mse', total_loss_sen.item()/(iteration+1))

In [33]:
best_val_loss = None
try:
    for epoch in range(1, epochs+1):
        epoch_start_time = time.time()
        training(False, train_loader)
        training(True, train_loader)
        evaluate(test_loader)
        print('-'*89)
except KeyboardInterrupt:
    print('-'*89)
    print('Existing from training early')

| epoch   1 |   100/   18 batches | ms/batch 64.03 | loss  0.37985
| epoch   1 |   200/   18 batches | ms/batch 60.23 | loss  0.32795
| epoch   1 |   300/   18 batches | ms/batch 59.59 | loss  0.30394
| epoch   1 |   400/   18 batches | ms/batch 60.43 | loss  0.29835
| epoch   1 |   500/   18 batches | ms/batch 59.54 | loss  0.29231
| epoch   1 |   600/   18 batches | ms/batch 60.85 | loss  0.28773
| epoch   1 |   700/   18 batches | ms/batch 59.66 | loss  0.29126
| epoch   1 |   800/   18 batches | ms/batch 61.18 | loss  0.27300
| epoch   1 |   900/   18 batches | ms/batch 60.07 | loss  0.27739
| epoch   1 |  1000/   18 batches | ms/batch 60.22 | loss  0.26793
| epoch   1 |  1100/   18 batches | ms/batch 61.83 | loss  0.27291
| epoch   1 |  1200/   18 batches | ms/batch 60.25 | loss  0.27563
| epoch   1 |  1300/   18 batches | ms/batch 60.42 | loss  0.27264
| epoch   1 |  1400/   18 batches | ms/batch 62.63 | loss  0.25508
| epoch   1 |  1500/   18 batches | ms/batch 60.54 | loss  0.2

In [34]:
#model save
torch.save(model.state_dict(), data_path+'exp4-id-feat-gene30-ALMANAC.p')

In [36]:
#model load
model.load_state_dict(torch.load(data_path+'exp4-id-feat-gene30-ALMANAC.p'))
model.eval()

Comb(
  (drugEncoder): DrugEncoder(
    (embed_comp): Embedding(48, 48, padding_idx=0)
    (encoderlayer): TransformerEncoderLayer(
      (self_attn): MultiheadAttention(
        (out_proj): NonDynamicallyQuantizableLinear(in_features=48, out_features=48, bias=True)
      )
      (linear1): Linear(in_features=48, out_features=2048, bias=True)
      (dropout): Dropout(p=0.1, inplace=False)
      (linear2): Linear(in_features=2048, out_features=48, bias=True)
      (norm1): LayerNorm((48,), eps=1e-05, elementwise_affine=True)
      (norm2): LayerNorm((48,), eps=1e-05, elementwise_affine=True)
      (dropout1): Dropout(p=0.1, inplace=False)
      (dropout2): Dropout(p=0.1, inplace=False)
    )
    (encoder): TransformerEncoder(
      (layers): ModuleList(
        (0): TransformerEncoderLayer(
          (self_attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=48, out_features=48, bias=True)
          )
          (linear1): Linear(in_features=48, 

# Evaluate the general model

In [37]:
def dcg_score(y_score, y_true, k):
    """
        https://www.kaggle.com/davidgasquez/ndcg-scorer
        y_true: np.array, size= [n_samples]
        y_score: np.array, size=[n_samples]
        k: int, rank
    """
    order = np.argsort(y_score)[::-1]
    y_true = np.take(y_true, order[:k])
    
    #gain = 2 ** y_true -1
    gain = y_true 
    
    discounts = np.log2(np.arange(len(y_true)) + 2)
    return np.sum(gain/discounts)

def evaluate_accuracy(data_loader):
    model.eval()
    
    syn_all=[]
    syn_true_all=[]
    ri1_all=[]
    ri1_true_all=[]
    ri2_all=[]
    ri2_true_all=[]
    
    #loss
    with torch.no_grad():
        for iteration, sample in enumerate(data_loader):
            d1=Variable(sample['d1'])
            d1_fp = Variable(sample['d1_fp'].float())
            d1_sm = Variable(sample['d1_sm'])
            d1_gn = Variable(sample['d1_gn'].float())

            d2=Variable(sample['d2'])
            d2_fp = Variable(sample['d2_fp'].float())
            d2_sm = Variable(sample['d2_sm'])
            d2_gn = Variable(sample['d2_gn'].float())

            cell = Variable(sample['cell'])
            c_ts = Variable(sample['c_ts'])
            c_ds = Variable(sample['c_ds'])
            c_gn = Variable(sample['c_gn'].float())

            syn_true = Variable(sample['syn'].float())
            ri_d1=Variable(sample['ri_d1'])
            ri_d2=Variable(sample['ri_d2'])


            if cuda:
                d1=d1.cuda()
                d1_fp=d1_fp.cuda()
                d1_sm=d1_sm.cuda()
                d1_gn=d1_gn.cuda()

                d2=d2.cuda()
                d2_fp=d2_fp.cuda()
                d2_sm=d2_sm.cuda()
                d2_gn=d2_gn.cuda()

                cell=cell.cuda()
                c_ts=c_ts.cuda()
                c_ds=c_ds.cuda()
                c_gn=c_gn.cuda()


            syn,ri1,ri2 = model((d1, d1_fp, d1_sm, d1_gn), (d2, d2_fp, d2_sm, d2_gn), (cell, c_ts,c_ds,c_gn) )
            
            syn_all.append(syn.data.cpu().numpy())
            syn_true_all.append(syn_true.numpy())
            
            ri1_all.append(ri1.data.cpu().numpy())
            ri1_true_all.append(ri_d1.numpy())
            
            ri2_all.append(ri2.data.cpu().numpy())
            ri2_true_all.append(ri_d2.numpy())
            
    return syn_all, syn_true_all, ri1_all, ri1_true_all, ri2_all, ri2_true_all


Evaluate synergy prediction

In [38]:
syn_all, syn_true_all, ri1_all, ri1_true_all, ri2_all, ri2_true_all= evaluate_accuracy(test_loader)

syn_all= [s.item() for syn in syn_all for s in syn]
syn_true_all = [s for syn in syn_true_all for s in syn]

In [39]:
#NDCG
dcg_score(syn_all,syn_true_all, k=20)/dcg_score(syn_true_all,syn_true_all, k=20)

1.0

In [40]:
#AUPRC
metrics.average_precision_score(syn_true_all,  1/(1 + np.exp(-np.array(syn_all))))

0.8597005380338557

In [41]:
#AUROC
metrics.roc_auc_score(syn_true_all,  1/(1 + np.exp(-np.array(syn_all))))

0.9626759388821085

Evaluate sensitivity prediction

In [42]:
ri1_all= [r.item() for ri in ri1_all for r in ri]
ri1_true_all = [r for ri in ri1_true_all for r in ri]

ri2_all= [r.item() for ri in ri2_all for r in ri]
ri2_true_all = [r for ri in ri2_true_all for r in ri]

ri_all=ri1_all+ri2_all
ri_true_all=ri1_true_all+ri2_true_all

In [43]:
#NDCG
dcg_score(ri_all,ri_true_all, k=20)/dcg_score(ri_true_all,ri_true_all, k=20)

0.3387952081956902

In [44]:
#AUC
metrics.roc_auc_score(ri_true_all,  1/(1 + np.exp(-np.array(ri_all))))

0.8651108739972198

# Transfer the general model to specific model

If you want to boost a bit more with general model's test set

In [45]:
#Use major's test set
training(False, test_loader)
training(True, test_loader)

| epoch  10 |   100/   18 batches | ms/batch 62.44 | loss  0.21246
| epoch  10 |   200/   18 batches | ms/batch 62.74 | loss  0.20853
| epoch  10 |   300/   18 batches | ms/batch 63.19 | loss  0.20654
| epoch  10 |   400/   18 batches | ms/batch 63.33 | loss  0.20849
| epoch  10 |   500/   18 batches | ms/batch 64.04 | loss  0.19935
| epoch  10 |   100/   18 batches | ms/batch 64.36 | loss  0.16313
| epoch  10 |   200/   18 batches | ms/batch 63.71 | loss  0.16199
| epoch  10 |   300/   18 batches | ms/batch 64.42 | loss  0.16235
| epoch  10 |   400/   18 batches | ms/batch 64.95 | loss  0.15352
| epoch  10 |   500/   18 batches | ms/batch 64.38 | loss  0.15460


## Freeze layers

Examine the layer's ID that we'd like to fix or free

In [46]:
for i, param in enumerate(model.parameters()):
    print(i, param.size(), param.requires_grad)

0 torch.Size([48, 48]) True
1 torch.Size([144, 48]) True
2 torch.Size([144]) True
3 torch.Size([48, 48]) True
4 torch.Size([48]) True
5 torch.Size([2048, 48]) True
6 torch.Size([2048]) True
7 torch.Size([48, 2048]) True
8 torch.Size([48]) True
9 torch.Size([48]) True
10 torch.Size([48]) True
11 torch.Size([48]) True
12 torch.Size([48]) True
13 torch.Size([144, 48]) True
14 torch.Size([144]) True
15 torch.Size([48, 48]) True
16 torch.Size([48]) True
17 torch.Size([2048, 48]) True
18 torch.Size([2048]) True
19 torch.Size([48, 2048]) True
20 torch.Size([48]) True
21 torch.Size([48]) True
22 torch.Size([48]) True
23 torch.Size([48]) True
24 torch.Size([48]) True
25 torch.Size([32, 167]) True
26 torch.Size([32]) True
27 torch.Size([32, 64]) True
28 torch.Size([32]) True
29 torch.Size([1, 1, 1, 48]) True
30 torch.Size([1]) True
31 torch.Size([352]) True
32 torch.Size([352]) True
33 torch.Size([176, 352]) True
34 torch.Size([176]) True
35 torch.Size([64, 176]) True
36 torch.Size([64]) True
37

In [47]:
release_after = 46

In [48]:
for i, param in enumerate(model.parameters()):
    if i>=release_after:
        param.requires_grad=True
    else:
        param.requires_grad=False

Prostate or bone

In [49]:
_train_loader_minor=_train_loader_prostate
_test_loader_minor=_test_loader_prostate
_test_minor=_test_prostate
#_train_loader_minor=_train_loader_bone
#_test_loader_minor=_test_loader_bone
#_test_minor=_test_bone

In [50]:
#Use minor's train set
try:
    for epoch in range(1, epochs+1):
        epoch_start_time = time.time()
        training(False, _train_loader_minor)
        training(True, _train_loader_minor)
        evaluate(_test_loader_minor)
        print('-'*89)
except KeyboardInterrupt:
    print('-'*89)
    print('Existing from training early')

syn mse 0.23522883967349403
sen_mse 0.054807305335998535
-----------------------------------------------------------------------------------------
syn mse 0.21754811939440274
sen_mse 0.05209772837789435
-----------------------------------------------------------------------------------------
syn mse 0.20951858319734273
sen_mse 0.05222006847983912
-----------------------------------------------------------------------------------------
syn mse 0.20513082805432772
sen_mse 0.05404499330018696
-----------------------------------------------------------------------------------------
syn mse 0.20166143618131938
sen_mse 0.05618089123776084
-----------------------------------------------------------------------------------------
syn mse 0.19788769671791478
sen_mse 0.05457201756929096
-----------------------------------------------------------------------------------------
syn mse 0.19574369882282458
sen_mse 0.053817065138565864
------------------------------------------------------------------

Evaluate synergy prediction

In [51]:
syn_all, syn_true_all, ri1_all, ri1_true_all, ri2_all, ri2_true_all= evaluate_accuracy(_test_loader_minor)

syn_all= [s.item() for syn in syn_all for s in syn]
syn_true_all = [s for syn in syn_true_all for s in syn]

In [52]:
#NDCG
dcg_score(syn_all,syn_true_all, k=20)/dcg_score(syn_true_all,syn_true_all, k=20)

1.0

In [53]:
#AUROC
metrics.roc_auc_score(syn_true_all,  1/(1 + np.exp(-np.array(syn_all))))

0.9606108567590311

In [54]:
#AUPRC
metrics.average_precision_score(syn_true_all,  1/(1 + np.exp(-np.array(syn_all))))

0.8047669798951004

Evaluate sensitivity prediction

In [55]:
ri1_all= [r.item() for ri in ri1_all for r in ri]
ri1_true_all = [r for ri in ri1_true_all for r in ri]

ri2_all= [r.item() for ri in ri2_all for r in ri]
ri2_true_all = [r for ri in ri2_true_all for r in ri]

ri_all=ri1_all+ri2_all
ri_true_all=ri1_true_all+ri2_true_all

In [56]:
#NDCG
dcg_score(ri_all,ri_true_all, k=20)/dcg_score(ri_true_all,ri_true_all, k=20)

0.06922125855611583

In [57]:
#AUC
metrics.roc_auc_score(ri_true_all,  1/(1 + np.exp(-np.array(ri_all))))

0.7822307029174376

# Select the top ranked drug combinations

In [58]:
syn_all_prob=1/(1 + np.exp(-np.array(syn_all)))
order = np.argsort(syn_all_prob)[::-1]
syn_true_all_order = np.take(syn_true_all, order[:20])

In [59]:
for k in range(20):
    comb=_test_minor[order[k]]
    print(codes['drugs'].idx2item[comb['d1']], ',', 
          codes['drugs'].idx2item[comb['d2']], ',', 
          codes['cell'].idx2item[comb['cell']],  ',', 
          codes['tissue'].idx2item[comb['c_ts']],  ',',
          codes['disease'].idx2item[comb['c_ds']], ',', 
          comb['ri_d1'], ',',
          comb['ri_d2'], ',',
          syn_true_all_order[k])


CHEMBL17639 , IXABEPILONE , DU-145 , prostate , c4863 , 0 , 0 , 1.0
ELOXATIN (TN) (SANOFI SYNTHELAB) , IXABEPILONE , DU-145 , prostate , c4863 , 0 , 0 , 1.0
IMATINIB , IXABEPILONE , DU-145 , prostate , c4863 , 0 , 0 , 1.0
ACTINOMYCIN D , IXABEPILONE , PC-3 , prostate , c4863 , 0 , 0 , 1.0
PLICAMYCIN , IXABEPILONE , DU-145 , prostate , c4863 , 0 , 0 , 1.0
QUINACRINE HYDROCHLORIDE , IXABEPILONE , DU-145 , prostate , c4863 , 0 , 0 , 1.0
FLUDARABINE BASE , IXABEPILONE , DU-145 , prostate , c4863 , 0 , 0 , 1.0
ELOXATIN (TN) (SANOFI SYNTHELAB) , IXABEPILONE , PC-3 , prostate , c4863 , 0 , 0 , 1.0
DOCETAXEL , IXABEPILONE , PC-3 , prostate , c4863 , 0 , 0 , 1.0
IMATINIB , IXABEPILONE , PC-3 , prostate , c4863 , 0 , 0 , 1.0
PRALATREXATE , IXABEPILONE , DU-145 , prostate , c4863 , 0 , 0 , 1.0
LETROZOLE , IXABEPILONE , DU-145 , prostate , c4863 , 0 , 0 , 1.0
RUXOLITINIB , CABAZITAXEL , DU-145 , prostate , c4863 , 0 , 0 , 1.0
CO-V , IXABEPILONE , DU-145 , prostate , c4863 , 0 , 0 , 1.0
HYDROXYUREA